In [1]:
import numpy as np;
import pandas as pd;
import plotly.express as px;


In [2]:
# Global parameters
NO_OF_DAYS = 75
modelParameter = {}
modelParameter["beta"] = 0.49  
modelParameter["tE"] = 5
modelParameter["tI"] = 5
modelParameter["tH"] = 9
modelParameter["mu"] = 0.001
modelParameter["cw"] = 15
modelParameter["ch"] = 5


print(modelParameter)


{'beta': 0.49, 'tE': 5, 'tI': 5, 'tH': 9, 'mu': 0.001, 'cw': 15, 'ch': 5}


In [3]:
# Load data
initDataDF = pd.read_csv("../data/tvm_init_data.csv")
#print(initDataDF)

cSparseMatrix = pd.read_csv("../output/cmatrix_results.csv")
#print(neighborDF)


In [4]:
## Differential equation
def deriv(y, Ni, contactRatio, modelParameter):
    Si, Ei, Ii, Hi, Ri = y
    dSdt = -modelParameter["beta"] * Si * contactRatio
    dEdt = (modelParameter["beta"] * Si * contactRatio) - (Ei/modelParameter["tE"])
    dIdt = (Ei/modelParameter["tE"]) -  (Ii/modelParameter["tI"])
    dHdt = (Ii/modelParameter["tI"]) - (Hi/modelParameter["tH"])
    dRdt = (Hi/modelParameter["tH"])
    return dSdt, dEdt, dIdt, dHdt, dRdt

In [5]:
def findSpatialContactRatio(name, NDict, yDict, cSparseMatrix, modelParameter):
    # TODO conver this to a matrix operation
    contactRatio = 0;
    Ci = cSparseMatrix[cSparseMatrix['iName']==name]
    cw = modelParameter["cw"]
    ch = modelParameter["ch"]
    for index, Cij in Ci.iterrows():
        jName = Cij['jName']
        cij = cw*Cij['Cij']

        if(jName == name):
            cij = (cw+ch)*Cij['Cij']
            
        Ij = yDict[jName][2]
        Nj = NDict[jName]
        contactRatio = contactRatio +  (cij*Ij/Nj)
    return contactRatio;
        

In [6]:
## next count 
def derivNext(y, rates) :
    S,E,I,H, R = y
    dSdt, dEdt, dIdt, dHdt, dRdt = rates
    S1 = S + dSdt
    E1 = E + dEdt
    I1 = I + dIdt
    H1 = H + dHdt
    R1 = R + dRdt
    if S1 <= 0:
        S1 = 0
    if E1 <= 0:
        E1 = 0
    if I1 <=0:
        I1 = 0
    if H1 <= 0:
        H1 = 0
    if R1 <= 0:
        R1 = 0
        
    return S1, E1, I1, H1, R1


In [ ]:
yDict = {};
NDict = {};
resultsDF = pd.DataFrame(columns=['name', 'day', 'S', 'E', 'I', 'H', 'R'], );

for index, row in initDataDF.iterrows():
    NDict[row['name']] = row['S'] + row['E'] + row['I'] + row['H'] + row['R']
    yDict[row['name']] = row['S'], row['E'], row['I'], row['H'], row['R'] 
    

for day in range(1,NO_OF_DAYS):
    for index, row in initDataDF.iterrows():
        name = row['name'];
        y = yDict[name]; 
        
        contactRatio = findSpatialContactRatio(name, NDict, yDict, cSparseMatrix, modelParameter);
        #print(name, "contactRatio = ", contactRatio)
        
        rates = deriv(y, NDict[name], contactRatio, modelParameter)
        yNext = derivNext(y, rates) 
        
        yDict[name] = yNext
        #print(name, yNext)
        resultsDF = resultsDF.append({'name': name, 'day': day, 'S': yNext[0], 'E': yNext[1], 'I': yNext[2], 'H': yNext[3], 'R': yNext[4]},ignore_index=True)


In [ ]:
resultsDF.head(10)

In [ ]:
resultsDF.to_csv("../output/results.csv")

In [ ]:
fig = px.scatter(resultsDF, x="day", y="H", color="name")
fig.show()